In [33]:
from nbdev.export import *
notebook2script()

Converted index.ipynb.
Converted rest.ipynb.
Converted services.ipynb.
Converted term.ipynb.


In [1]:
# default_exp services
# export
from pathlib import Path
import yaml
import os
import time

In [34]:
# export

quick_scrum_folder = 'quick-scrum'

def validate_project(path):
    required = ['project.yaml', 'sandbox', 'backlog', 'users','docs']
    return all([(f/quick_scrum_folder/name).exists() for name in required])

def scan_for_projects(path):
    project_folders = [f.parent.absolute() for f in path.glob('**/{quick_scrum_folder}')]
    return [f for f in project_folders if validate_project(f)]
        
def find_project(path):
    qs_path = path/'quick-scrum'
    if qs_path.exists():
        return qs_path
    elif path == path.root:
        return None
    else:
        return find_project(path.parent)

In [35]:
# export

def reg_project(path):
    qs_config = Path('~/.config/qs.yaml')
    qs_config.parent.mkdir(parents=True)
    if qs_config.exists():
        

def init_project(path, project):
    existing_projects = scan_for_projects(path)
    if len(existing_projects):
        paths = ','.join([str(p) for p in existing_projects])
        raise FileExistsError(f'A quick-scrum project already exists in {paths}')
    
    qs_path = path/quick_scrum_folder
    required = ['sandbox', 'backlog', 'docs','users']
    [(qs_path/n).mkdir(parents=True) for n in required]
    with (qs_path/'project.yaml').open('w') as f:
        c = {**project, 'created': time.time()}
        yaml.dump(c, f)

In [2]:
class Project:
    def __init__(self, path):
        self.path = path
        
    def find_project(self, path):
        
    
    def exists():
        
    def 

IndentationError: expected an indented block (<ipython-input-2-374a37c8c59a>, line 8)

In [15]:
# export
class Scrum:
    def __init__(self, home_folder):
        self.home = Path(home_folder)
    
    def get_projects(self):
        "Get all the projects available in the home folder"
        return [p.name for p in (self.home/'projects').iterdir() if p.is_dir()]
    
    def get_project(self, project_name):
        "Get the project info"
        p = self.home/'projects'/project_name/'project.yaml'
        if p.exists():
            with p.open() as f:
                return yaml.load(f, Loader=yaml.FullLoader)
        else:
            return None
    
    def new_project(self, project):
        n = project['name']
        p = self.home/'projects'/n
        if p.exists():
            raise ValueError(f'Project {n} already exist')
        
        [f.mkdir(parents=True) for f in [p, p/'backlog', p/'sandbox']]
        with (p/'project.yaml').open('w') as f:
            c = {**project, 'created': time.time()}
            yaml.dump(c, f)
        
    def get_sprints(self, project_name):
        "Get all the sprints in the project prj"
        return [p.name for p in (self.home/'projects'/project_name)
                .glob('sprint-*') if p.is_dir()]
        
    def new_sprint(self, project_name):
        "Create a new sprint"
        ss = sorted(self.get_sprints(project_name))
        if ss:
            last = ss[-1]
            (_,n) = last.split('-')
            s = f'sprint-{int(n)+1}'
        else:
            s = f'sprint-1'
        (self.home/'projects'/project_name/s).mkdir()
        return s
        
    def get_stories(self, project_name, store):
        "Get all the stories name for a store, i.e. backlog, sprint..."
        pp = self.home/'projects'/project_name/store
        pp = sorted(pp.glob('*.yaml'), key=os.path.getmtime)
        return [p.stem for p in pp]

    def get_story(self, project_name, store, story_name):
        "Get the content of a story"
        p = self.home/'projects'/project_name/store/f'{story_name}.yaml'
        if not p.exists():
            return None
        with p.open() as f:
            c = yaml.load(f, Loader=yaml.FullLoader)
            return {'name': story_name, **c}

    def del_story(self, project_name, store, story_name):
        p = self.home/'projects'/project_name/store/f'{story_name}.yaml'
        if p.exists():
            p.unlink()
            return True
        else:
            return False
        
    def set_story(self, project_name, store, story):
        name = story['name']
        p = self.home/'projects'/project_name/store/f'~{name}.yaml'
        with p.open('w') as f:
            yaml.dump(story, f)
        p = p.replace(self.home/'projects'/project_name/store/f'{name}.yaml')
        return p.stem
        
    def touch_story(self, project_name, store, story):
        name = story['name']
        p = self.home/'projects'/project_name/store/f'~{name}.yaml'
        p.touch()
        
    def move_story(self, project_name, source, target, story_name):
        "Move a story to a different container"
        p = self.home/'projects'/project_name/source/f'{story_name}.yaml'
        if p.exist():
            p.replace(self.home/'projects'/project_name/target/f'{name}.yaml')
            return True
        else:
            return False

In [9]:
s = Scrum('../data')

In [10]:
s.get_projects()

['new Proj 34h', 'new Proj', 'new Proj 34', 'new Proj 3', 'test-prj']

In [12]:
s.new_project({'name':'test-prj'})

ValueError: Project test-prj already exist

In [7]:
s.new_sprint('test-prj')

'sprint-7'

In [8]:
s.get_stories('test-prj','sprint-2')

[]

In [27]:
s.get_logs('te2st-prj')

[]

In [ ]:
st = s.get_story('test-prj','sandbox','define-the-interface')

In [ ]:
s.set_story('test-prj','sandbox',{
    'name': 'a new story',
    'description': 'with some description'
})

In [ ]:
s = [2]

In [ ]:
if s: print('dd')